<H1> PART ONE CREATING A NOTEBOOK

In [14]:
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import requests # Library for website scraping

!pip install BeautifulSoup4
from bs4 import BeautifulSoup

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

from sklearn.cluster import KMeans # import k-means from clustering stage

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [16]:
#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, "html.parser")

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [17]:
canada_df = canada_df[canada_df['Borough'] != 'Not assigned'] #Remove Boroughs that are 'Not assigned'
canada_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5> GROUPING NEIGHBORHOOD WITH SAME POSTAL CODE TOGETHER

In [24]:
canada_df = canada_df.groupby(["Postal Code", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [26]:
print("Shape: ", canada_df.shape)

Shape:  (103, 3)


<H1> PART TWO ADDING LOCATION

In [35]:
df_geo_coor = pd.read_csv("Geospatial_Coordinates.csv") #Adding location CSV file
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
df_toronto = pd.merge(canada_df, df_geo_coor, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


<H1> EXPLORE AND CLUSTER TORONTO NEIGHBORHOOD

In [39]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="'toronto'_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [41]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [43]:
# add markers to map
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [45]:
# Exploring North York Borough of Toronto
df_toronto_NY = df_toronto[df_toronto['Borough'].str.contains("North York")].reset_index(drop=True)
df_toronto_NY.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Hillcrest Village,43.803762,-79.363452
1,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,North York,Bayview Village,43.786947,-79.385975
3,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


In [46]:
map_toronto_NY = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(
        df_toronto_NY['Latitude'], 
        df_toronto_NY['Longitude'], 
        df_toronto_NY['Borough'], 
        df_toronto_NY['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_NY)  

map_toronto_NY

In [47]:
CLIENT_ID = 'U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH' # your Foursquare ID
CLIENT_SECRET = 'HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH
CLIENT_SECRET:HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG


In [48]:
neighborhood_name = df_toronto_NY.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'Hillcrest Village'.


In [49]:
neighborhood_latitude = df_toronto_NY.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_NY.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


In [50]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH&client_secret=HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG&v=20180605&ll=43.8037622,-79.3634517&radius=500&limit=100'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [51]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [52]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695


In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH&client_secret=HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG&v=20180605&ll=43.8037622,-79.3634517&radius=500&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
nearby_venues_NY = getNearbyVenues(names=df_toronto_NY['Neighborhood'],
                                   latitudes=df_toronto_NY['Latitude'],
                                   longitudes=df_toronto_NY['Longitude']
                                  )
nearby_venues_NY.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports


In [58]:
nearby_venues_NY.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",5,5,5,5,5,5
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",5,5,5,5,5,5
Don Mills,10,10,10,10,10,10
Downsview,20,20,20,20,20,20
"Fairview, Henry Farm, Oriole",5,5,5,5,5,5
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,5,5,5,5,5,5


In [60]:
#Unique categories from all venues
print('There are {} uniques categories.'.format(len(nearby_venues_NY['Venue Category'].unique())))

There are 5 uniques categories.


In [61]:
# one hot encoding
toronto_NY_onehot = pd.get_dummies(nearby_venues_NY[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_NY_onehot['Neighborhood'] = nearby_venues_NY['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_NY_onehot.columns[-1]] + list(toronto_NY_onehot.columns[:-1])
toronto_NY_onehot = toronto_NY_onehot[fixed_columns]

toronto_NY_onehot.head()

,Neighborhood,Athletics & Sports,Dog Run,Golf Course,Mediterranean Restaurant,Pool
0,Hillcrest Village,0,0,1,0,0
1,Hillcrest Village,0,0,0,0,1
2,Hillcrest Village,0,0,0,1,0
3,Hillcrest Village,0,1,0,0,0
4,Hillcrest Village,1,0,0,0,0


In [62]:
toronto_NY_grouped = toronto_NY_onehot.groupby('Neighborhood').mean().reset_index()
toronto_NY_grouped.head()

,Neighborhood,Athletics & Sports,Dog Run,Golf Course,Mediterranean Restaurant,Pool
0,"Bathurst Manor, Wilson Heights, Downsview North",0.2,0.2,0.2,0.2,0.2
1,Bayview Village,0.2,0.2,0.2,0.2,0.2
2,"Bedford Park, Lawrence Manor East",0.2,0.2,0.2,0.2,0.2
3,Don Mills,0.2,0.2,0.2,0.2,0.2
4,Downsview,0.2,0.2,0.2,0.2,0.2


<h5> From results, it looks like they are equally popular around the neighborhood

In [63]:
# set number of clusters
kclusters = 2

toronto_NY_grouped_clustering = toronto_NY_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [66]:
# add clustering labels
toronto_NY_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_NY_merged = df_toronto_NY

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_NY_merged = toronto_NY_merged.join(toronto_NY_grouped.set_index('Neighborhood'), on='Neighborhood')

toronto_NY_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Athletics & Sports,Dog Run,Golf Course,Mediterranean Restaurant,Pool
0,North York,Hillcrest Village,43.803762,-79.363452,0,0.2,0.2,0.2,0.2,0.2
1,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,0.2,0.2,0.2,0.2,0.2
2,North York,Bayview Village,43.786947,-79.385975,0,0.2,0.2,0.2,0.2,0.2
3,North York,"York Mills, Silver Hills",43.757490,-79.374714,0,0.2,0.2,0.2,0.2,0.2
4,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,0,0.2,0.2,0.2,0.2,0.2


In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_NY_merged['Latitude'], 
        toronto_NY_merged['Longitude'], 
        toronto_NY_merged['Neighborhood'], 
        toronto_NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [70]:
toronto_NY_merged.loc[toronto_NY_merged['Cluster Labels'] == 0, toronto_NY_merged.columns[[1] + list(range(5, toronto_NY_merged.shape[1]))]]

,Neighborhood,Athletics & Sports,Dog Run,Golf Course,Mediterranean Restaurant,Pool
0,Hillcrest Village,0.2,0.2,0.2,0.2,0.2
1,"Fairview, Henry Farm, Oriole",0.2,0.2,0.2,0.2,0.2
2,Bayview Village,0.2,0.2,0.2,0.2,0.2
3,"York Mills, Silver Hills",0.2,0.2,0.2,0.2,0.2
4,"Willowdale, Newtonbrook",0.2,0.2,0.2,0.2,0.2
5,"Willowdale, Willowdale East",0.2,0.2,0.2,0.2,0.2
6,York Mills West,0.2,0.2,0.2,0.2,0.2
7,"Willowdale, Willowdale West",0.2,0.2,0.2,0.2,0.2
8,Parkwoods,0.2,0.2,0.2,0.2,0.2
9,Don Mills,0.2,0.2,0.2,0.2,0.2
